For example, you could add settings to the chat prompt from the previous article with the following code

In [8]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs 
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.8.0"
#r "nuget: Microsoft.SemanticKernel.PromptTemplates.Handlebars, 1.8.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core,1.8.0-alpha"
using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.PromptTemplates.Handlebars;
using Microsoft.SemanticKernel.PromptTemplates;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Plugins.Core;

//Create Kernel builder
var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();
kernel.ImportPluginFromType<ConversationSummaryPlugin>();

       var history = """
                  <message role="user">I hate sending emails, no one ever reads them.</message>
                  <message role="assistant">I'm sorry to hear that. Messages may be a better way to communicate.</message>
                  """;
                  string request = "I want to send an email to the marketing team celebrating their recent milestone.";
// Create a template for chat with settings
var chat = kernel.CreateFunctionFromPrompt(
    new PromptTemplateConfig()
    {
        Name = "Chat",
        Description = "Chat with the assistant.",
        Template = @"{{ConversationSummaryPlugin.SummarizeConversation $history}}
                    User: {{$request}}
                    Assistant: ",
        TemplateFormat = "semantic-kernel",
        InputVariables =
        [
            new() { Name = "history", Description = "The history of the conversation.", IsRequired = false, Default = "" },
            new() { Name = "request", Description = "The user's request.", IsRequired = true }
        ],
        ExecutionSettings =
        {
            {
                "default",
                new OpenAIPromptExecutionSettings()
                {
                    MaxTokens = 1000,
                    Temperature = 0
                }
            },
            {
                "gpt-3.5-turbo", new OpenAIPromptExecutionSettings()
                {
                    ModelId = "gpt-3.5-turbo-0613",
                    MaxTokens = 4000,
                    Temperature = 0.2
                }
            },
            {
                "gpt-4",
                new OpenAIPromptExecutionSettings()
                {
                    ModelId = "gpt-4-1106-preview",
                    MaxTokens = 8000,
                    Temperature = 0.3
                }
            }
        }
    }
);

 // Get chat response
            var chatResult = kernel.InvokeStreamingAsync<StreamingChatMessageContent>(
                chat,
                new()
                {
                    { "request", request },
                    { "history", history }
                }
            );

            // Stream the response
            string message = "";
            await foreach (var chunk in chatResult)
            {
                if (chunk.Role.HasValue)
                {
                    Console.Write(chunk.Role + " > ");
                }

                message += chunk;
                Console.Write(chunk);
            }
            Console.WriteLine();


Installed Packages Microsoft.SemanticKernel, 1.8.0 Microsoft.SemanticKernel.Plugins.Core, 1.8.0-alpha Microsoft.SemanticKernel.PromptTemplates.Handlebars, 1.8.0

Error: (22,29): error SKEXP0050: 'Microsoft.SemanticKernel.Plugins.Core.ConversationSummaryPlugin' is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.